**Задание 1.**

Будем делать рекомендации не всем и сразу, а конкретному пользователю.


1. Возьмём все оценки, которые поставил пользователь  U 
2. По фильмам  MU , которые он оценил, получим всех пользователей, которые ставили этим фильмам оценки.
3. Скорее всего и на этом этапе список получится большим (для случайного пользователя у меня получилось 1.7 миллионов строк). Например, оставить только пользователей, у которых много общих фильмов.
4. Для каждого пользователя посчитаем похожесть (например, корреляцию).
5. Отсортируем пользователей по похожести.
6. Оставим только  k  самых похожих пользователей.
7. Оценить качество этой рекомендации. Идея тут такая: разделить исходные данные на train и test. Получить предсказания на train и проверить их на test.



In [ ]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install PyDrive

In [ ]:
from google.colab import auth
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
from pydrive.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1dGbV43VYV8yzHeZHxvIsVIGYzyzukjWd"}) 
downloaded.GetContentFile('ratings.csv')
rating=pd.read_csv('ratings.csv')

In [ ]:
downloaded = drive.CreateFile({'id':"1W0L7Vr02vT5PSGWGMk4BLtSrg-aFaP75"}) 
downloaded.GetContentFile('movies.csv')
movies=pd.read_csv('movies.csv')

In [ ]:
data = pd.merge(movies,rating)

**Решение.**

Выбираем случайного пользователя.

In [ ]:
all_users = rating['userId'].value_counts()
user = all_users.sample(1).index
user

Int64Index([71664], dtype='int64')

Получаем фильмы, которые он посмотрел.

In [ ]:
user_movies = data[data.userId == user[0]]
user_movies = user_movies.drop(['genres', 'timestamp', 'userId'], axis = 1)
user_movies.shape

(100, 3)

Разделим выборку этих фильмов на train и test.

In [ ]:
from sklearn.model_selection import train_test_split

user_movies_train, user_movies_test = train_test_split(user_movies, test_size=0.33, random_state=42)

In [ ]:
user_movies_train.shape

(67, 3)

Теперь найдем пользователей, которые смотрели эти фильмы.

In [ ]:
new_data = data[data.movieId.isin(user_movies_train.movieId)]
new_data = new_data.drop(['genres', 'timestamp'], axis = 1)

In [ ]:
users_group = new_data.groupby(['userId'])
sorted_users_group = sorted(users_group,  key=lambda x: len(x[1]), reverse=True)

len(sorted_users_group)

147887

In [ ]:
sorted_users_group = sorted_users_group[:50000]

Далее нужно пройтись по всем пользователям и отсортировать их по похожести и оставить  k  самых похожих.

In [ ]:
from scipy.stats import pearsonr

In [ ]:
pearsonCorDict = {}
for i in range(len(sorted_users_group)):
  person2 = users_group.get_group(sorted_users_group[i][0]).sort_values(by='movieId')
  temp = user_movies[user_movies['movieId'].isin(users_group.get_group(sorted_users_group[i][0])['movieId'])]
  person1 = temp.sort_values(by='movieId')
  if pearsonr(person1.rating, person2.rating)[0] >= 0.65: # оставляем только пользователей с корреляцией выше 0.65
    pearsonCorDict[i] = pearsonr(person1.rating, person2.rating)[0]

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)
topUsers

,similarityIndex,userId
0,1.000000,0
1627,0.954721,46636
1735,0.947204,47932
1349,0.943005,43116
1222,0.924211,41389
...,...,...
301,0.650110,19529
684,0.650109,31432
1879,0.650084,49541
780,0.650006,33675


Теперь у нас есть похожие пользователи. Получим оценки, которые они ставили за все фильмы.

In [ ]:
topUsersRating=topUsers.merge(data, left_on='userId', right_on='userId', how='inner')
topUsersRating.sample(10)

,similarityIndex,userId,movieId,title,genres,rating,timestamp
300941,0.650959,28367,315,"Specialist, The (1994)",Action|Drama|Thriller,4.0,843166893
72020,0.758735,37505,43928,Ultraviolet (2006),Action|Fantasy|Sci-Fi|Thriller,3.5,1540199622
153211,0.704484,26932,437,Cops and Robbersons (1994),Comedy,3.0,837602563
124922,0.718657,48539,4015,"Dude, Where's My Car? (2000)",Comedy|Sci-Fi,2.0,1045857900
14035,0.842701,49384,4489,Coming to America (1988),Comedy|Romance,2.5,1483223024
221501,0.675129,35327,1587,Conan the Barbarian (1982),Action|Adventure|Fantasy,3.5,1487164900
21944,0.826788,49403,103253,Elysium (2013),Action|Drama|Sci-Fi|IMAX,1.0,1393038552
245583,0.667983,32691,4344,Swordfish (2001),Action|Crime|Drama,3.0,1567141384
254099,0.665469,43812,51662,300 (2007),Action|Fantasy|War|IMAX,3.0,1510167503
194883,0.686170,27950,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,4.0,1353727568


Уберём лишние столбцы и получим взвешенную оценку за фильм.

In [ ]:
topUsersRating = topUsersRating.drop(['genres', 'timestamp'], axis = 1)
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']


Теперь нужно сгруппировать по фильмам и получить сумму взвешенных оценок.

In [ ]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']

In [ ]:
recommendation_df = pd.DataFrame()
recommendation_df['score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df = recommendation_df.sort_values(by='score', ascending=False)

In [ ]:
recommendation_df2 = recommendation_df[recommendation_df.score > 4.5]

In [ ]:
recommendation_df2 = recommendation_df2.drop(['movieId'], axis = 1)

In [ ]:
recommended_movies=recommendation_df2.merge(movies, how = 'inner', on = 'movieId')

In [ ]:
recommended_movies

,movieId,score,title,genres
0,159123,5.000000,Catching Out (2003),(no genres listed)
1,96807,5.000000,Breathless (2012),Thriller
2,7230,5.000000,"Silencers, The (1966)",Action|Comedy
3,1075,5.000000,"Sexual Life of the Belgians, The (Vie sexuelle...",Comedy|Romance
4,192911,5.000000,A.X.L. (2018),Sci-Fi
...,...,...,...,...
592,180249,4.502345,"No Date, No Signature (2017)",Drama
593,26578,4.502220,"Sacrifice, The (Offret - Sacraficatio) (1986)",Drama
594,6031,4.501826,Imitation of Life (1959),Drama|Romance
595,198185,4.500000,Twin Peaks (1989),Drama|Mystery


Теперь нужно оценить качество этой рекомендации.

In [ ]:
recommended_movies = recommendation_df.drop(['movieId'], axis = 1)
recommended_movies_test=recommended_movies.merge(movies, how = 'inner', on = 'movieId')

In [ ]:
recommendation_test = user_movies_test.merge(recommended_movies_test, how = 'inner', on = ['title'])

In [ ]:
recommendation_test.head()

,movieId_x,title,rating,movieId_y,score,genres
0,539,Sleepless in Seattle (1993),3.0,539,3.516061,Comedy|Drama|Romance
1,318,"Shawshank Redemption, The (1994)",5.0,318,4.429634,Crime|Drama
2,431,Carlito's Way (1993),4.0,431,3.737990,Crime|Drama
3,277,Miracle on 34th Street (1994),3.0,277,3.251887,Drama
4,267,Major Payne (1995),3.0,267,2.959121,Comedy


Воспользуемся здесь метрикой MSE (Mean Squared Error). 
В идеальном случае, когда все ответы предсказаны точно, MSE будет равна нулю.

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(recommendation_test.rating, recommendation_test.score)

0.29058566884455067

Как мы видим, идея рекомендаций на основе предпочтений "ближайших соседей" позволяет предсказать то, как пользователь оценит рекомендованный ему фильм. 

Однако, точность такой оценки колеблется в зависимости от:

1) Количества похожих пользователей, которых мы анализируем. Здесь мы брали всех с коэффициентом похожести от 0.65, и чем больше количество таких пользователей, тем точнее предсказание.

2) Количества фильмов, просмотренных пользователей, если их слишком много или слишком мало, размер ошибки возрастает.

Данные рекомендации достаточно неплохие, если анализировать ограниченную выборку фильмов, однако стоит учитывать свойства контента (теги, жанры, мб режиссеры и актеры и прочее) для того, чтобы повышать качество рекомендаций.